<a href="https://colab.research.google.com/github/mnansary/pyF2O/blob/master/colab_gen_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   mount google drive
*   change working directory to git repo
*   Check TF version
*    TPU check



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
cd /content/gdrive/My\ Drive/PROJECTS/F2O/pyF2O

In [0]:
!pip3 install tensorflow==1.13.1

## TPU Check

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

tf.__version__

# UNET generator Model Training

## Compile Model

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from models.generators import unet
model=unet()
model.summary()
model.compile(optimizer=Adam(lr=0.01), loss=binary_crossentropy)

## Convert Model

In [0]:
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

def convert_model_TPU(model):
  return tf.contrib.tpu.keras_to_tpu_model(model,
                                           strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                               tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
                                               )
                                           )

model=convert_model_TPU(model)

## Batched Data 
* For **Validation** the 5 random **Train_x.tfrecords** were moved to new folder called **Eval** manually
* The **tfrecords** should now have three folders: **Train**, **Test**and **Eval**   

In [0]:
class EVAL_FLAGS:
    TFRECORDS_DIR   = '/content/gdrive/My Drive/PROJECTS/F2O/tfrecords/Eval/' # @param
    BATCH_SIZE      = 128  # @param
    NUM_EPOCHS      = 500  # @param
    IMAGE_DIM       = 256  # @param
    SHUFFLE_BUFFER  = 200  # @param

print('Eval FLAGS Loaded')    

In [0]:
class TRAIN_FLAGS:
    TFRECORDS_DIR   = '/content/gdrive/My Drive/PROJECTS/F2O/tfrecords/Train/' # @param
    BATCH_SIZE      = 128  # @param
    NUM_EPOCHS      = 500  # @param
    IMAGE_DIM       = 256  # @param
    SHUFFLE_BUFFER  = 200  # @param

print('Train FLAGS Loaded')    

In [0]:
from models.utils import get_batched_dataset
def training_data():
  return get_batched_dataset(TRAIN_FLAGS)
def validation_data():
  return get_batched_dataset(EVAL_FLAGS)


## Train

In [0]:
history=model.fit(training_data(),validation_data=(validation_data()),epochs=TRAIN_FLAGS.NUM_EPOCHS,batch_size=TRAIN_FLAGS.BATCH_SIZE, verbose=1)

## Save Model Weights

In [0]:
import os
model.save_weights(os.path.join(os.getcwd(),'models','model_weights','UNET.h5'))

## Plot Training Histoty

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('LOSS')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')